In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
import unicodedata
import re

# project_path = "/content/drive/MyDrive/Hackathon_2025/datasets/raw"
project_path = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw"
dpe_path = os.path.join(project_path, "dpe_sample_2M.csv")

dpe = pd.read_csv(dpe_path, low_memory=False)
print(dpe.shape)
dpe.head()

Mounted at /content/drive
(2000000, 30)


,conso_5_usages_ef,code_insee_ban,deperditions_planchers_bas,numero_voie_ban,conso_5_usages_ep,date_fin_validite_dpe,adresse_brut,conso_5_usages_par_m2_ep,date_etablissement_dpe,conso_5_usages_par_m2_ef,...,date_reception_dpe,adresse_ban,date_derniere_modification_dpe,etiquette_ges,qualite_isolation_enveloppe,type_installation_chauffage,code_postal_ban,surface_habitable_logement,etiquette_dpe,_score
0,7643.1,75113,0.0,50,8248.7,2032-02-06,50 rue du disque,257.0,2022-02-07,238.0,...,2022-02-08,50 Rue du Disque 75013 Paris,2024-06-25,D,insuffisante,collectif,75013.0,32.0,D,NaN
1,9978.0,54437,17.8,12,22949.4,2032-04-23,12 rue de la Rouotte,309.0,2022-04-24,134.0,...,2022-04-25,12 Rue de la Rouotte 54160 Pulligny,2025-01-20,B,insuffisante,individuel,54160.0,74.2,E,NaN
2,7969.2,75107,0.0,74,8350.1,2032-06-05,74 RUE DE SEVRES,216.6,2022-06-06,206.7,...,2022-06-07,74 Rue de Sèvres 75007 Paris,2024-06-25,D,insuffisante,individuel,75007.0,38.6,D,NaN
3,5932.8,44205,4.2,28,13645.4,2032-06-28,28 rue du Calvaire,238.0,2022-06-29,103.0,...,2022-06-30,28 Rue du Calvaire 44390 Les Touches,2024-02-06,B,bonne,NaN,44390.0,57.2,D,NaN
4,3639.9,93047,0.0,41,8371.7,2032-01-31,41 RUE HENRI BARBUSSE,259.0,2022-02-01,112.0,...,2022-02-02,41 Rue Henri Barbusse 93370 Montfermeil,2024-06-25,B,insuffisante,individuel,93370.0,32.2,D,NaN


2. On ne garde que ce qui servira pour le match avec Enedis + la conso et quelques infos de base.

In [ ]:
colonnes_utiles = [
    "numero_dpe",
    "adresse_ban", "adresse_brut",
    "numero_voie_ban", "nom_rue_ban", "nom_commune_ban",
    "code_postal_ban", "code_insee_ban",
    "surface_habitable_logement",
    "etiquette_dpe", "etiquette_ges",
    "conso_5_usages_par_m2_ep", "conso_5_usages_ep",
    "conso_5_usages_par_m2_ef", "conso_5_usages_ef",
    "type_batiment",
    "periode_construction", "annee_construction",
    "date_etablissement_dpe", "date_fin_validite_dpe",
    "date_reception_dpe", "date_visite_diagnostiqueur",
    "date_derniere_modification_dpe",
    "qualite_isolation_enveloppe",
    "qualite_isolation_murs",
    "qualite_isolation_plancher_bas",
    "qualite_isolation_menuiseries",
    "type_installation_chauffage",
    "type_installation_ecs"
]

dpe = dpe[colonnes_utiles].copy()
dpe.head()

,numero_dpe,adresse_ban,adresse_brut,numero_voie_ban,nom_rue_ban,nom_commune_ban,code_postal_ban,code_insee_ban,surface_habitable_logement,etiquette_dpe,...,conso_5_usages_par_m2_ef,conso_5_usages_ef,type_batiment,periode_construction,annee_construction,date_etablissement_dpe,date_fin_validite_dpe,date_reception_dpe,date_visite_diagnostiqueur,date_derniere_modification_dpe
0,2275E0252200Y,50 Rue du Disque 75013 Paris,50 rue du disque,50,Rue du Disque,Paris,75013.0,75113,32.0,D,...,238.0,7643.1,appartement,1948-1974,1948.0,2022-02-07,2032-02-06,2022-02-08,2022-02-02,2024-06-25
1,2254E0866216D,12 Rue de la Rouotte 54160 Pulligny,12 rue de la Rouotte,12,Rue de la Rouotte,Pulligny,54160.0,54437,74.2,E,...,134.0,9978.0,appartement,avant 1948,1947.0,2022-04-24,2032-04-23,2022-04-25,2022-04-24,2025-01-20
2,2275E1242094U,74 Rue de Sèvres 75007 Paris,74 RUE DE SEVRES,74,Rue de Sèvres,Paris,75007.0,75107,38.6,D,...,206.7,7969.2,appartement,avant 1948,1948.0,2022-06-06,2032-06-05,2022-06-07,2022-06-06,2024-06-25
3,2244E1489464V,28 Rue du Calvaire 44390 Les Touches,28 rue du Calvaire,28,Rue du Calvaire,Les Touches,44390.0,44205,57.2,D,...,103.0,5932.8,maison,avant 1948,NaN,2022-06-29,2032-06-28,2022-06-30,2022-06-28,2024-02-06
4,2293E0210795D,41 Rue Henri Barbusse 93370 Montfermeil,41 RUE HENRI BARBUSSE,41,Rue Henri Barbusse,Montfermeil,93370.0,93047,32.2,D,...,112.0,3639.9,appartement,avant 1948,1947.0,2022-02-01,2032-01-31,2022-02-02,2022-01-31,2024-06-25


3. Filtrer un minimum les lignes : garder seulement maison / appartement et supprimer quelques valeurs aberrantes très grossières

In [3]:
# 3.1. Garder seulement logements
dpe = dpe[dpe["type_batiment"].isin(["maison", "appartement"])].copy()

# 3.2. Nettoyage très simple des valeurs num.
dpe = dpe[
    (dpe["surface_habitable_logement"].between(9, 500)) &
    (dpe["conso_5_usages_par_m2_ep"].between(50, 800))
].copy()

# 3.3. Supprimer les lignes sans classe DPE
dpe = dpe[dpe["etiquette_dpe"].notna()]

print(dpe.shape)

(1955344, 30)


4. Parser les dates et l’année de construction

In [4]:
date_cols = [
    "date_etablissement_dpe",
    "date_fin_validite_dpe",
    "date_reception_dpe",
    "date_visite_diagnostiqueur",
    "date_derniere_modification_dpe",
]

for c in date_cols:
    dpe[c] = pd.to_datetime(dpe[c], errors="coerce")

# année de construction en entier (nullable)
dpe["annee_construction"] = (
    dpe["annee_construction"].round().astype("Int64")
)

5. Créer une adresse propre pour le join : adresse_clean

In [5]:
def normalize_text(s: str) -> str:
    if pd.isna(s):
        return ""
    s = str(s).lower()
    # enlever accents
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    # garder uniquement chiffres/lettres -> espaces
    s = re.sub(r"[^0-9a-z]+", " ", s)
    # enlever espaces multiples
    s = " ".join(s.split())
    return s

# adresse source = BAN si dispo, sinon adresse_brut
dpe["adresse_source"] = dpe["adresse_ban"].fillna(dpe["adresse_brut"])

# adresse normalisée, prête pour jointure avec Enedis
dpe["adresse_clean"] = dpe["adresse_source"].apply(normalize_text)

dpe[["adresse_source", "adresse_clean"]].head()

,adresse_source,adresse_clean
0,50 Rue du Disque 75013 Paris,50 rue du disque 75013 paris
1,12 Rue de la Rouotte 54160 Pulligny,12 rue de la rouotte 54160 pulligny
2,74 Rue de Sèvres 75007 Paris,74 rue de sevres 75007 paris
3,28 Rue du Calvaire 44390 Les Touches,28 rue du calvaire 44390 les touches
4,41 Rue Henri Barbusse 93370 Montfermeil,41 rue henri barbusse 93370 montfermeil


In [6]:
dpe["conso_theorique_logement_kwh_an"] = dpe["conso_5_usages_ef"]
dpe["conso_theorique_kwh_m2_an"] = dpe["conso_5_usages_par_m2_ef"]

In [7]:
ordre = list("ABCDEFG")
mapping = {lettre: i+1 for i, lettre in enumerate(ordre)}

dpe["etiquette_dpe_num"] = dpe["etiquette_dpe"].map(mapping)
dpe["etiquette_ges_num"] = dpe["etiquette_ges"].map(mapping)

In [8]:
output_path = os.path.join("/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean", "dpe_sample_2M_clean.csv")
dpe.to_csv(output_path, index=False)
print("Fichier sauvegardé :", output_path)

Fichier sauvegardé : /content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/dpe_sample_2M_clean.csv
